In [ ]:
import re
import polars as pl
import requests
from bs4 import BeautifulSoup
from datetime import datetime

Traceback (most recent call last):
  File "/root/.vscode-remote/extensions/ms-python.python-2024.20.0-linux-x64/python_files/python_server.py", line 130, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 6, in <module>
ModuleNotFoundError: No module named 'pytz'



In [ ]:
response = requests.get("https://www.flysfo.com/passengers/parking")
soup = BeautifulSoup(response.content, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en" prefix="og: https://ogp.me/ns#">
 <head>
  <title>
   Parking at SFO | San Francisco International Airport
  </title>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="Parking at SFO. Book online for the best rates and convenient parking to the terminals!" name="description">
   <meta content="|" name="abstract"/>
   <meta content="onsite SFO parking, SFO parking, long term SFO parking, domestic SFO, international SFO parking, parking deals SFO, parking discounts, parking sale SFO, san francisco airport parking" name="keywords"/>
   <meta content="US-CA" name="geo.region"/>
   <meta content="San Francisco International Airport" name="geo.placename"/>
   <meta content="37.61564176706988,-122.39180242454718" name="geo.position"/>
   <link href="https://www.flysfo.com/node/9364" rel="shortlink"/>
   <link href="https://www.flysfo.com/passengers/parking" rel="canonical"/>
   <meta content="Copyright ©2024 All 

In [ ]:

gc = soup.find(class_="garageCard")
print(gc.attrs)

{'class': ['garageCard']}


In [ ]:

garage_card_title = gc.find("div", class_="garageCard__title")
print(garage_card_title.text)

 Domestic Garage


In [ ]:
gc_all = soup.find_all(class_="garageCard")


In [ ]:
current_time_pst = datetime.now()
current_date = current_time_pst.strftime("%Y-%m-%d")
current_time = current_time_pst.strftime("%H:%M:%S")

dat = pl.DataFrame(
    {
        "date": [current_date for _ in gc_all],
        "time": [current_time for _ in gc_all],
        "garage": [_.find("div", class_="garageCard__title").text.strip() for _ in gc_all],
        "spots": [
            int(
                re.search(
                    r"\d+", _.find("p", class_="garageCard__infos__spots").text.strip()
                ).group()
            )
            for _ in gc_all
        ],
        "occupancy": [
            int(
                re.search(
                    r"\d+", _.find("text", class_="percentage").text.strip()
                ).group()
            )
            for _ in gc_all
        ],
    }
)
print(dat)

shape: (5, 5)
┌────────────┬──────────┬────────────────────────┬───────┬───────────┐
│ date       ┆ time     ┆ garage                 ┆ spots ┆ occupancy │
│ ---        ┆ ---      ┆ ---                    ┆ ---   ┆ ---       │
│ str        ┆ str      ┆ str                    ┆ i64   ┆ i64       │
╞════════════╪══════════╪════════════════════════╪═══════╪═══════════╡
│ 2024-12-12 ┆ 04:15:31 ┆ Domestic Garage        ┆ 1180  ┆ 76        │
│ 2024-12-12 ┆ 04:15:31 ┆ International Garage A ┆ 578   ┆ 65        │
│ 2024-12-12 ┆ 04:15:31 ┆ International Garage G ┆ 493   ┆ 65        │
│ 2024-12-12 ┆ 04:15:31 ┆ ParkFAST               ┆ 106   ┆ 55        │
│ 2024-12-12 ┆ 04:15:31 ┆ Long Term Garage       ┆ 2652  ┆ 60        │
└────────────┴──────────┴────────────────────────┴───────┴───────────┘
